In [1]:
#package imports and magic words

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from collections import defaultdict, Counter

### Guiding questions: 
Some questions about the data to motivate exploration.
 - General Time Series
     - Has average runtime changed over time?
     - Has average budget or average revenue changed over time?
 - Director and cast impact on revenue/popularity
     - What directors and cast members are associated with high revenue (revenue_adj)?
     - What directors and cast members are associated with high popularity (and/or vote_average)?
     - Are there frequent occurences of directors and cast?
 - Genres
     - What genres are associated with high revenues and high popularity?
     - Are certain genres released at certain times of the year?
     - Have certain genres increased in occurence over time?

In [2]:
#read in dataset and dedupe
df_total = pd.read_csv('tmdb-movies.csv')
df_total.drop_duplicates(inplace = True)

#Look at general structure of data after load 
df_total.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,101199955.5,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,183999919.0,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,174799923.1,1.385749e+09


In [3]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10865 entries, 0 to 10865
Data columns (total 21 columns):
id                      10865 non-null int64
imdb_id                 10855 non-null object
popularity              10865 non-null float64
budget                  10865 non-null int64
revenue                 10865 non-null int64
original_title          10865 non-null object
cast                    10789 non-null object
homepage                2936 non-null object
director                10821 non-null object
tagline                 8041 non-null object
keywords                9372 non-null object
overview                10861 non-null object
runtime                 10865 non-null int64
genres                  10842 non-null object
production_companies    9835 non-null object
release_date            10865 non-null object
vote_count              10865 non-null int64
vote_average            10865 non-null float64
release_year            10865 non-null int64
budget_adj              1

## Data Assessment

- Total number of records: 10865 (non-null)
- Features missing records: homepage, tagline, keywords, production_companies, director, genres, overview, and cast.
- Columns containing multiple variables/values: cast, genres, production_companies.
- Column release_date should be a datetime object type.
- Column release_year should be a category type.
- Some records of budget, revenue, budget_adj, and revenue_adj are listed as 0. These values should probably be listed as nulls (np.nan).
- Column id is an indicator and should be string type.

## Cleaning

- Split Cast into invidual columns (cast_1, cast_2, etc).
- Split genres into individual colmuns (genre_1, genre_2, etc).
- Split production_companies into individual colmuns (prod_comp_1, prod_comp_2, etc).
- Set new columns for genre, cast, and procution_companies to category type.
- Convert release_date to datetime object.
- Convert release_year to category type.
- Convert 0s for budget, revenue, budget_adj, and revenue_adj, to np.nan.
- Convert id to string.

## Additonal Variables

- Add revenue/budget ratio.
- Add category for revenue levels.

## Data Cleaning

### 1. Save off temp data frame for cleaning

In [4]:
df_clean = df_total.copy()

### 2. Make type conversions and value conversions

- Use .astype, .replace, .to_datetime as needed to make changes to values and types.
- Verify changes with .info.

In [8]:
df_clean.id = df_clean.id.astype('str')
df_clean.release_year = df_clean.release_year.astype('category')
df_clean.release_date = pd.to_datetime(df_clean.release_date)
financials = ['budget', 'revenue', 'budget_adj', 'revenue_adj']
for col in financials:
    df_clean[col].replace(0, np.nan, inplace = True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10865 entries, 0 to 10865
Data columns (total 21 columns):
id                      10865 non-null object
imdb_id                 10855 non-null object
popularity              10865 non-null float64
budget                  5169 non-null float64
revenue                 4849 non-null float64
original_title          10865 non-null object
cast                    10789 non-null object
homepage                2936 non-null object
director                10821 non-null object
tagline                 8041 non-null object
keywords                9372 non-null object
overview                10861 non-null object
runtime                 10865 non-null int64
genres                  10842 non-null object
production_companies    9835 non-null object
release_date            10865 non-null datetime64[ns]
vote_count              10865 non-null int64
vote_average            10865 non-null float64
release_year            10865 non-null category
budget_adj 

### 3. Add columns for splitting qualitative nested data

- Found that null records contained the string 'nan'. Converted to np.nan for better operation.

In [63]:
#value conversion for nans
df_clean.cast.replace('nan', np.nan, inplace = True)

#find number of new columns to make
cast_cnt = df_clean[df_clean.cast.notnull()].cast.str.split('|').apply(len)
print('Max number of cast members listed in a column: ' +
      str(max(np.unique(cast_cnt))) +
      '. \nUse this for the additional columns.')

#make new columns filled with nans
for member in range(max(cast_cnt)):
    df_clean['cast_' + str(member + 1)] = [np.nan] * df_clean.shape[0]
    
#split cast column into new columns


Max number of cast members listed in a column: 5. 
Use this for the additional columns.


In [64]:
df_clean.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,vote_count,vote_average,release_year,budget_adj,revenue_adj,cast_1,cast_2,cast_3,cast_4,cast_5
0,135397,tt0369610,32.985763,150000000.0,1.513529e+09,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,5562,6.5,2015,137999939.3,1.392446e+09,NaN,NaN,NaN,NaN,NaN
1,76341,tt1392190,28.419936,150000000.0,3.784364e+08,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,6185,7.1,2015,137999939.3,3.481613e+08,NaN,NaN,NaN,NaN,NaN
2,262500,tt2908446,13.112507,110000000.0,2.952382e+08,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,2480,6.3,2015,101199955.5,2.716190e+08,NaN,NaN,NaN,NaN,NaN
3,140607,tt2488496,11.173104,200000000.0,2.068178e+09,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,5292,7.5,2015,183999919.0,1.902723e+09,NaN,NaN,NaN,NaN,NaN
4,168259,tt2820852,9.335014,190000000.0,1.506249e+09,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,2947,7.3,2015,174799923.1,1.385749e+09,NaN,NaN,NaN,NaN,NaN
